In [1]:
import os
import cv2
import imutils
import numpy as np
from PIL import Image

C:\Python310\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Python310\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Python310\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [5]:
def ReshapeImages(FolderName):
  # Reshape Images into 100x89
  files = os.listdir(FolderName)
  for i in files:
    img = Image.open(FolderName + "/" + i) 
    img = img.resize((100, int((float((100 / float(img.size[0]))) * float(img.size[1])))), Image.ANTIALIAS)
    img.save(FolderName + "/" + i)

In [6]:
def CreateData(FolderPath, ImageName = "palm", start_index = 0, Number_of_Images = 100, Number_of_Frames = 30):
  sr = False #Flag to start Recording
  checker = None
  frames, image_index= 0, start_index
  camera = cv2.VideoCapture(0)
  pt,  pb = 10, 225
  pr, pl = 350, 590

  while(1):
      (g, f) = camera.read()
      if g:
          f = cv2.flip(imutils.resize(f, width=700), 1)
          (height, width) = f.shape[:2]
          img_cpy = f.copy()
          gray = cv2.GaussianBlur(cv2.cvtColor(f[pt:pb, pr:pl], cv2.COLOR_BGR2GRAY), (7, 7), 0)
          

          if frames < Number_of_Frames:
              if checker is None:
                   checker = gray.copy().astype("float")
              else:
                  cv2.accumulateWeighted(gray, checker, 0.5)
          else:
              y = cv2.threshold(cv2.absdiff(checker.astype("uint8"), gray), 25, 255, cv2.THRESH_BINARY)[1]
              (x, _) = cv2.findContours(y.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
              if 0 != len(x):
                h = (y, max(x, key=cv2.contourArea))
                if h is not None:
                  (t, s) = h
                  cv2.drawContours(img_cpy, [(pr, pt) + s], -1, (0, 0, 255))
                  if sr == True:
                      cv2.imwrite(FolderPath + "/" + ImageName + str(image_index) + '.png', t)
                      image_index = image_index + 1
                  cv2.imshow("Gray Image", t)

          cv2.rectangle(img_cpy, (pl, pt), (pr, pb), (0, 255, 0), 2)
          frames = frames + 1

          cv2.imshow("Data Collection Window", img_cpy)
          k = cv2.waitKey(1) & 0xFF

          if  start_index + Number_of_Images < image_index:
              break
          elif k == ord("a"):
              sr = True
      else:
          break
  camera.release()
  cv2.destroyAllWindows()
  ReshapeImages(FolderPath)

In [8]:
#this will create 100 images data
CreateData("kara/thums_up","thums_up",0)